<a href="https://colab.research.google.com/github/Incredible88/Ezio-/blob/master/Finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get start

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
!pip install chardet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import chardet
with open('/content/FOMC_minutes.csv', 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']

In [ ]:
encoding

'Windows-1252'

# Load minutes data

In [ ]:
text = pd.read_csv('/content/FOMC_minutes.csv',encoding='Windows-1252')

In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Month     382 non-null    int64 
 1   Day       382 non-null    int64 
 2   Year      382 non-null    int64 
 3   date      382 non-null    object
 4   sentence  382 non-null    object
dtypes: int64(3), object(2)
memory usage: 15.0+ KB


# load Finbert 

In [14]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
finbert = pipeline("text-classification", model=finbert, tokenizer=tokenizer)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.9 MB/s eta 0:00:00


## Using spacy to split sentences

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

sentences = []
for text in tqdm(text['sentence']):
    doc = nlp(text)
    for sent in doc.sents:
        sentences.append(sent.text)


100%|██████████| 382/382 [02:47<00:00,  2.28it/s]


In [ ]:
len(sentences)

0

# Predict the original dataset


In [ ]:
nlp(sentences_list[1])

[{'label': 'Neutral', 'score': 0.9999871253967285}]

In [ ]:
results = []
for sentence in tqdm(sentences):
    doc = finbert(sentence)
    label = doc[0]['label']
    score = doc[0]['score']
    results.append({'sentence': sentence, 'label': label, 'score': score})

df = pd.DataFrame(results)

100%|██████████| 32330/32330 [1:47:26<00:00,  5.02it/s]


In [ ]:
df.head()

,sentence,label,score
0,The Manager of the System Open Market Account ...,Neutral,0.999998
1,There were no open market operations in foreig...,Neutral,0.999987
2,The Manager also reported on developments in d...,Neutral,0.999989
3,"By unanimous vote, the Committee ratified thes...",Neutral,0.999968
4,The information reviewed at this meeting sugge...,Positive,1.000000


In [ ]:
df.to_csv('FOMC_results_1.csv', index=False)

# Text Simplification (sentiment focus)

**Step1.**  
To remove comma when two paragraph should be together. For Example:  
 **---**  
However, the apparent pickup in longer-term expectations, while worrisome, was relatively small  
⇓  
However, the apparent pickup in longer-term expectations, while worrisome was relatively small 
  
**Step2.**
If there is a transition such as **although** and **while**, the focus will be on another paragraph, and if there is a **but**, the focus will be on the paragraph containing the but. For example:

 **---**  
**Although** some scattered signs of cooling of the housing sector had emerged, the pace of construction activity and sales remained brisk.  
⇓  
the pace of construction activity and sales remained brisk.


---


Starts of new single-family homes dropped back somewhat in October from September's very strong pace, **but** permit issuance remained elevated.   
⇓  
permit issuance remained elevated.




In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

import spacy

nlp = spacy.load("en_core_web_sm")

def remove_comma(sentence):
    doc = nlp(sentence)
    indices = []
    for i, token in enumerate(doc):
        if token.dep_ == "punct":
            try:               
                next_token = doc[i+1]
                if next_token.dep_ == "ROOT" or next_token.dep_ == "conj":
                    indices.append(i)
            except IndexError:
                pass
    if not indices:
        return sentence
    else:
        parts = []
        last_idx = 0
        for idx in indices:
            parts.append(doc[last_idx:idx].text.strip())

            last_idx = idx+1
        parts.append(doc[last_idx:].text.strip())
        return " ".join(parts)


In [26]:
remove_comma("The personal saving rate--while still slightly negative--moved up in October.")

'The personal saving rate--while still slightly negative moved up in October.'

In [36]:
def sentiment_focus(sentence):
    doc = nlp(sentence)
    focus = ""
    focus_changed = 1
    for sent in doc.sents:
        sent_tokens = [token for token in sent]
        for token in sent_tokens:
            if token.lower_ == "although" or token.lower_ == "while":
                try:
                    comma_index_back = [token1.i for token1 in doc[token.i:] if token1.text == ','][0]
                except IndexError:
                    try:
                      comma_index_front = [token1.i for token1 in doc[:token.i] if token1.text == ','][-1]
                    except IndexError:
                      return str(doc).strip(),focus_changed
                    focus = doc[:comma_index_front].text
                    return str(focus).strip(),focus_changed
                try:
                      comma_index_front = [token1.i for token1 in doc[:token.i] if token1.text == ','][-1]
                except IndexError:
                  focus = doc[comma_index_back+1:].text
                  return str(focus).strip(),focus_changed
                focus = doc[:comma_index_front].text+doc[comma_index_back:].text
                return str(focus).strip(),focus_changed

    for index, token in enumerate(doc[:-1]):
        if token.lower_ == "but":
            focus = doc[index + 1:]
            return str(focus).strip(),focus_changed
    focus_changed = 0 
    return str(doc).strip(),focus_changed

In [22]:
remove_comma("Participants noted that, while the pass-through of higher energy and other commodity prices to prices of core goods and services had remained subdued, there were continuing upside risks to inflation from these sources.")

'Participants noted that, while the pass-through of higher energy and other commodity prices to prices of core goods and services had remained subdued, there were continuing upside risks to inflation from these sources.'

In [23]:
sentiment_focus("Participants noted that, while the pass-through of higher energy and other commodity prices to prices of core goods and services had remained subdued, there were continuing upside risks to inflation from these sources.")

('Participants noted that', 1)

In [ ]:
finbert("However, the apparent pickup in longer-term expectations")

[{'label': 'Positive', 'score': 0.9998661279678345}]

In [34]:
sentiment_focus(" Starts of new single-family homes dropped back somewhat in October from September's very strong pace, but permit issuance remained elevated")

('permit issuance remained elevated', 1)

In [40]:
sentiment_focus("Spending on cars and light trucks increased somewhat in July after a lackluster pace in the second quarter but apparently weakened in August.")

('apparently weakened in August.', 1)

In [ ]:
sentiment_focus('Although some scattered signs of cooling of the housing sector had emerged, the pace of construction activity and sales remained brisk.')

'the pace of construction activity and sales remained brisk.'

In [ ]:
sentiment_focus("Spreads of yields on corporate bonds over those on comparable-maturity Treasury securities widened somewhat, while those on speculative-grade issues rose by more.")

('Spreads of yields on corporate bonds over those on comparable-maturity Treasury securities widened somewhat',
 1)

In [42]:
finbert("apparently weakened in August.")

[{'label': 'Negative', 'score': 0.9999364614486694}]

In [ ]:
finbert("Overall consumer prices jumped in March because of higher energy prices, while core prices rose a bit more rapidly than in earlier months.")

[{'label': 'Positive', 'score': 0.9938077330589294}]

In [ ]:
finbert("small time deposits and retail money market mutual funds again contracted at a rapid pace in response to the low yields on those assets")

[{'label': 'Negative', 'score': 0.7514410614967346}]

In [ ]:
finbert('permit issuance remained elevated')

[{'label': 'Negative', 'score': 0.8672457337379456}]

In [ ]:
finbert('the pace of construction activity and sales remained brisk.')

[{'label': 'Positive', 'score': 0.3631136417388916}]

## The same sentence after focus processing

Actually works well

In [ ]:
finbert("In their discussion of major sectors of the economy, meeting participants noted that, while light vehicle sales had slowed in the fall, consumer spending outside the auto sector appeared to have remained vigorous")

[{'label': 'Negative', 'score': 0.9846646189689636}]

In [ ]:
finbert("consumer spending outside the auto sector appeared to have remained vigorous")

[{'label': 'Positive', 'score': 0.8260715007781982}]

# Processing focus sentiment sentences

In [ ]:
df.head()

,sentence,label,score
0,The Manager of the System Open Market Account ...,Neutral,0.999998
1,There were no open market operations in foreig...,Neutral,0.999987
2,The Manager also reported on developments in d...,Neutral,0.999989
3,"By unanimous vote, the Committee ratified thes...",Positive,1.000000
4,"Industrial production rebounded, and employmen...",Positive,0.999996


In [2]:
df = pd.read_csv('/content/FOMC_results_1.csv')

In [38]:
import spacy
from tqdm.auto import tqdm
tqdm.pandas()

df["sentence_simple"] = df["sentence"].progress_apply(remove_comma)

  0%|          | 0/32330 [00:00<?, ?it/s]

In [39]:
df[['sentence_simple', 'focus_changed']] = df['sentence_simple'].progress_apply(sentiment_focus).apply(pd.Series)

df['focus_ornot'] = df['focus_changed'].apply(lambda x: 1 if x else 0)

df.drop('focus_changed', axis=1, inplace=True)

  0%|          | 0/32330 [00:00<?, ?it/s]

In [8]:
df.head()

,sentence,label,score,sentence_simple,focus_ornot
0,The Manager of the System Open Market Account ...,Neutral,0.999998,The Manager of the System Open Market Account ...,0
1,There were no open market operations in foreig...,Neutral,0.999987,There were no open market operations in foreig...,0
2,The Manager also reported on developments in d...,Neutral,0.999989,The Manager also reported on developments in d...,0
3,"By unanimous vote, the Committee ratified thes...",Neutral,0.999968,"By unanimous vote, the Committee ratified thes...",0
4,The information reviewed at this meeting sugge...,Positive,1.000000,The information reviewed at this meeting sugge...,0


## Predict with Finbert(only select changed sentences)

In [43]:
df.head()

,sentence,label,score,sentence_simple,focus_ornot
0,The Manager of the System Open Market Account ...,Neutral,0.999998,The Manager of the System Open Market Account ...,0
1,There were no open market operations in foreig...,Neutral,0.999987,There were no open market operations in foreig...,0
2,The Manager also reported on developments in d...,Neutral,0.999989,The Manager also reported on developments in d...,0
3,"By unanimous vote, the Committee ratified thes...",Neutral,0.999968,"By unanimous vote, the Committee ratified thes...",0
4,The information reviewed at this meeting sugge...,Positive,1.000000,The information reviewed at this meeting sugge...,0


In [46]:
df_change = df.loc[df['focus_ornot'] == 1]

In [47]:
df_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4222 entries, 6 to 32318
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentence         4222 non-null   object 
 1   label            4222 non-null   object 
 2   score            4222 non-null   float64
 3   sentence_simple  4222 non-null   object 
 4   focus_ornot      4222 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 197.9+ KB


In [48]:
df_change.head()

,sentence,label,score,sentence_simple,focus_ornot
6,Although some scattered signs of cooling of th...,Negative,0.645584,the pace of construction activity and sales re...,1
12,With employment rising but the average workwee...,Negative,0.999218,the average workweek of production or nonsuper...,1
23,Real disposable personal income was bolstered ...,Positive,1.000000,Real disposable personal income was bolstered ...,1
25,The personal saving rate--while still slightly...,Negative,0.852820,The personal saving rate--while still slightly...,1
27,Starts of new single-family homes dropped back...,Negative,0.950385,permit issuance remained elevated.,1


In [49]:
tqdm.pandas()

df_change['finbert_result'] = df_change['sentence_simple'].progress_apply(lambda x: finbert(x))

df_change['label_new'] = df_change['finbert_result'].apply(lambda x: x[0]['label'])
df_change['score_new'] = df_change['finbert_result'].apply(lambda x: x[0]['score'])

df_change.drop('finbert_result', axis=1, inplace=True)

  0%|          | 0/4222 [00:00<?, ?it/s]

<ipython-input-49-907bcaf9f350>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_change['finbert_result'] = df_change['sentence_simple'].progress_apply(lambda x: finbert(x))
<ipython-input-49-907bcaf9f350>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_change['label_new'] = df_change['finbert_result'].apply(lambda x: x[0]['label'])
<ipython-input-49-907bcaf9f350>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [52]:
df_change.head()

,sentence,label,score,sentence_simple,focus_ornot,label_new,score_new
6,Although some scattered signs of cooling of th...,Negative,0.645584,the pace of construction activity and sales re...,1,Positive,0.363114
12,With employment rising but the average workwee...,Negative,0.999218,the average workweek of production or nonsuper...,1,Negative,0.997591
23,Real disposable personal income was bolstered ...,Positive,1.000000,Real disposable personal income was bolstered ...,1,Positive,1.000000
25,The personal saving rate--while still slightly...,Negative,0.852820,The personal saving rate--while still slightly...,1,Negative,0.674692
27,Starts of new single-family homes dropped back...,Negative,0.950385,permit issuance remained elevated.,1,Negative,0.691221


In [51]:
df_change.to_csv('training_data.csv',index = False)

In [11]:
df_change1 = df.loc[df.sentence != df.sentence_simple]

In [19]:
df_change1 = df_change.loc[df_change.label != df_change.label_new]

In [20]:
df_change1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1250 entries, 42 to 32318
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentence         1250 non-null   object 
 1   label            1250 non-null   object 
 2   score            1250 non-null   float64
 3   sentence_simple  1250 non-null   object 
 4   focus_ornot      1250 non-null   int64  
 5   label_new        1250 non-null   object 
 6   score_new        1250 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 78.1+ KB


In [ ]:
df_change1.head()

,sentence,label,score,sentence_simple,focus_ornot,label_new,score_new
5,Although some scattered signs of cooling of th...,Negative,0.645584,the pace of construction activity and sales re...,1,Positive,0.363114
33,Foreign industrialized economies expanded robu...,Positive,1.000000,especially in transportation services.,1,Negative,0.492485
38,Longer-term inflation expectations also edged ...,Positive,0.970137,stayed a touch above the narrow range observed...,1,Neutral,0.946409
46,"In these circumstances, the Committee believed...",Neutral,0.929808,noted that it would respond to changes in econ...,1,Negative,0.516684
55,Although hurricane-related rebuilding would bo...,Positive,0.593083,"especially in the near term, this stimulus inc...",1,Negative,0.998140


# Fine tuning FinBERT



In [17]:
!pip install transformers
!pip install datasets
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import transformers
torch.__version__, transformers.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.1 MB/s eta 0:00:00


('2.0.0+cu118', '4.28.1')

In [4]:
torch.cuda.is_available()


True

In [7]:
df = pd.read_csv('/content/training_data.csv') 
df.head()

,sentence,label,score,sentence_simple,focus_ornot,label_new,score_new
0,Although some scattered signs of cooling of th...,Negative,0.645584,the pace of construction activity and sales re...,1,Positive,0.363114
1,With employment rising but the average workwee...,Negative,0.999218,the average workweek of production or nonsuper...,1,Negative,0.997591
2,Real disposable personal income was bolstered ...,Positive,1.000000,Real disposable personal income was bolstered ...,1,Positive,1.000000
3,The personal saving rate--while still slightly...,Negative,0.852820,The personal saving rate--while still slightly...,1,Negative,0.674692
4,Starts of new single-family homes dropped back...,Negative,0.950385,permit issuance remained elevated.,1,Negative,0.691221


In [20]:
df['label'] = df['label'].replace({'Neutral': 0, 'Positive': 1, 'Negative': 2})
df.head()

,sentence,label
0,Although some scattered signs of cooling of th...,1
1,With employment rising but the average workwee...,2
2,Real disposable personal income was bolstered ...,1
3,The personal saving rate--while still slightly...,2
4,Starts of new single-family homes dropped back...,2


## preparing training/validation/testing

In [12]:
df = df[['sentence', 'label_new']].rename(columns={'label_new': 'label'})

In [13]:
df.head()

,sentence,label
0,Although some scattered signs of cooling of th...,Positive
1,With employment rising but the average workwee...,Negative
2,Real disposable personal income was bolstered ...,Positive
3,The personal saving rate--while still slightly...,Negative
4,Starts of new single-family homes dropped back...,Negative


In [21]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(3419, 2) (423, 2) (380, 2)


## load FinBERT pretrained model
The pretrained FinBERT model path on Huggingface is https://huggingface.co/yiyanghkust/finbert-pretrain


In [15]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

## prepare dataset for fine-tuning

In [22]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/3419 [00:00<?, ? examples/s]

Map:   0%|          | 0/380 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

## define training options

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                        
        args=args,                  
        train_dataset=dataset_train,         
        eval_dataset=dataset_val,           
        compute_metrics=compute_metrics
)

trainer.train()  

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.483213,0.807895
2,No log,0.371464,0.847368
3,No log,0.412142,0.855263
4,No log,0.415011,0.863158
5,0.310400,0.463843,0.863158


TrainOutput(global_step=535, training_loss=0.29532779800557646, metrics={'train_runtime': 378.8394, 'train_samples_per_second': 45.125, 'train_steps_per_second': 1.412, 'total_flos': 1124480969015040.0, 'train_loss': 0.29532779800557646, 'epoch': 5.0})

## evaluate on testing set

In [24]:
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.5003162622451782,
 'test_accuracy': 0.8486997635933806,
 'test_runtime': 2.8429,
 'test_samples_per_second': 148.793,
 'test_steps_per_second': 4.925}

## save the fine-tuned model

In [25]:
trainer.save_model('finbert-sentiment/')